In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from numpy import genfromtxt

dataframe = genfromtxt('train.csv', delimiter=',', skip_header=True)


In [2]:
y, X = dataframe[:,0], dataframe[:, 1:]

In [3]:
# Split it into train and validation
n_train = int(0.6 * len(y))

y_train, X_train, y_test, X_test = y[: n_train], X[: n_train, :], y[n_train:], X[n_train:, :]

In [4]:
X_test.shape

(16800, 784)

In [5]:
import tensorflow as tf

input_ = tf.placeholder(dtype=tf.float32, shape=(None, X_train.shape[1]))
labels = tf.placeholder(dtype=tf.int32, shape=(None, 1))


weight_input_hidden = tf.Variable(tf.random_normal(shape=(int(input_.shape[1]), 128), mean=0.0, stddev=0.1, dtype=tf.float32))
bias_input_hidden = tf.Variable(tf.zeros(weight_input_hidden.shape[1]))

weight_hidden_output = tf.Variable(tf.random_normal(shape=(128, 10), mean=0.0, stddev=0.1, dtype=tf.float32))
bias_hidden_output = tf.Variable(tf.zeros(weight_hidden_output.shape[1]))

print(input_)
print(labels)
print(weight_input_hidden)
print(bias_input_hidden)
print(weight_hidden_output)
print(bias_hidden_output)

weight_dict = {'layer0': weight_input_hidden, 'layer1': weight_hidden_output}
bias_dict = {'layer0': bias_input_hidden, 'layer1': bias_hidden_output}

Tensor("Placeholder:0", shape=(?, 784), dtype=float32)
Tensor("Placeholder_1:0", shape=(?, 1), dtype=int32)
<tf.Variable 'Variable:0' shape=(784, 128) dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=(128,) dtype=float32_ref>
<tf.Variable 'Variable_2:0' shape=(128, 10) dtype=float32_ref>
<tf.Variable 'Variable_3:0' shape=(10,) dtype=float32_ref>


In [6]:
def nn():
    hidden_input = tf.add(tf.matmul(input_, weight_dict['layer0']), bias_dict['layer0'])
    hidden_output = tf.nn.relu(hidden_input)
    
    output = tf.add(tf.matmul(hidden_input, weight_dict['layer1']), bias_dict['layer1'])
    
    # Define the cost
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output, labels=tf.one_hot(labels,depth=10)))
    
    # Apply an optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)
    
    return cost, optimizer, output

In [7]:
def get_next_batch(batch_size, X, y):
    for i in range(int(y.shape[0]/batch_size)-1):
        yield X[i: i+batch_size, :] , y[i: i+batch_size]

In [16]:
def get_accuracy(logits, output, data):
        # Find the accuracy
        correct_prediction = tf.equal(tf.argmax(logits, 1), output)
        # Calculate accuracy
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    
        return accuracy.eval(feed_dict={input_: data, labels: output.reshape([-1,1])})

In [20]:
cost, optimizer, output = nn()

# Initialize tensorflow session
init = tf.global_variables_initializer()

epochs = 10
batch_size = 64
num_batches = int(X_train.shape[0]/batch_size)
n_train = int(len(y_train)*0.8)

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training cycle
    for epoch in range(epochs):
        # Shuffle data for cross validation
        indices = np.array(range(len(y_train)))
        np.random.shuffle(indices)
        
        train_indices = indices[: n_train]
        val_indices = indices[n_train:]
        
        y_train_fold, X_train_fold, y_val_fold, X_val_fold = y_train[train_indices], X_train[train_indices, :], \
        y_train[val_indices], X_train[val_indices, :]
        
        # Loop over all batches
        for x,y in get_next_batch(batch_size, X_train_fold, y_train_fold):
            sess.run(optimizer, feed_dict={input_:x, labels:y.reshape([-1,1])})
            
        c = sess.run(cost, feed_dict={input_: x, labels: y.reshape([-1,1])})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c))
        
        # Find train accuracy
        print("Train Accuracy:", get_accuracy(output, y_train_fold, X_train_fold))
        
        # Find the validation accuracy
        print("Validation Accuracy:", get_accuracy(output, y_val_fold, X_val_fold))
        print()
    
    print("Optimization Finished!")
    
    
    # Find test accuracy
    print("Test Accuracy:", get_accuracy(output, y_test, X_test))

Epoch: 0001 cost= 0.132104725
Train Accuracy: 0.764236
Validation Accuracy: 0.759127

Epoch: 0002 cost= 0.000000000
Train Accuracy: 0.778224
Validation Accuracy: 0.774206

Epoch: 0003 cost= 0.000000000
Train Accuracy: 0.758978
Validation Accuracy: 0.757143

Epoch: 0004 cost= 0.000000000
Train Accuracy: 0.78006
Validation Accuracy: 0.775595

Epoch: 0005 cost= 0.000000002
Train Accuracy: 0.781151
Validation Accuracy: 0.775

Epoch: 0006 cost= 0.000000000
Train Accuracy: 0.775546
Validation Accuracy: 0.766468

Epoch: 0007 cost= 0.000000000
Train Accuracy: 0.776488
Validation Accuracy: 0.768651

Epoch: 0008 cost= 0.000000000
Train Accuracy: 0.776538
Validation Accuracy: 0.768651

Epoch: 0009 cost= 0.000000000
Train Accuracy: 0.776587
Validation Accuracy: 0.768651

Epoch: 0010 cost= 0.000000000
Train Accuracy: 0.776587
Validation Accuracy: 0.768651

Optimization Finished!
Test Accuracy: 0.77119
